# Исследование надёжности заёмщиков

По заданию кредитного отдела банка нам необходимо выяснить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

Банка предоставил нам статистику о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга**

# Подготовительные действия

In [1]:
# импортируем необходимые библиотеки
from IPython.display import display
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

# отключаем предупреждения фильтрации
import warnings
warnings.filterwarnings("ignore")

<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Откроем-файл-с-данными-и-изучим-общую-информацию" data-toc-modified-id="Откроем-файл-с-данными-и-изучим-общую-информацию-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Откроем файл с данными и изучим общую информацию</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков-" data-toc-modified-id="Обработка-пропусков--2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обработка пропусков <a id="fill_gaps"></a></a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков-в-столбце-days_employed" data-toc-modified-id="Обработка-пропусков-в-столбце-days_employed-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Обработка пропусков в столбце <code>days_employed</code></a></span></li><li><span><a href="#Обработка-пропусков-в-столбце-dob_years" data-toc-modified-id="Обработка-пропусков-в-столбце-dob_years-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Обработка пропусков в столбце <code>dob_years</code></a></span></li><li><span><a href="#Обработка-пропусков-в-столбце-children" data-toc-modified-id="Обработка-пропусков-в-столбце-children-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Обработка пропусков в столбце <code>children</code></a></span></li><li><span><a href="#Обработка-пропусков-в-столбце-total_income" data-toc-modified-id="Обработка-пропусков-в-столбце-total_income-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Обработка пропусков в столбце <code>total_income</code></a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Замена-типа-данных" data-toc-modified-id="Замена-типа-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Замена типа данных</a></span></li><li><span><a href="#Обработка-дубликатов" data-toc-modified-id="Обработка-дубликатов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обработка дубликатов</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Категоризация-данных" data-toc-modified-id="Категоризация-данных-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Категоризация данных</a></span></li></ul></li><li><span><a href="#Ответим-на-интересующие-банк-вопросы" data-toc-modified-id="Ответим-на-интересующие-банк-вопросы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Ответим на интересующие банк вопросы</a></span><ul class="toc-item"><li><span><a href="#Есть-ли-зависимость-между-наличием-детей-и-возвратом-кредита-в-срок?" data-toc-modified-id="Есть-ли-зависимость-между-наличием-детей-и-возвратом-кредита-в-срок?-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Есть ли зависимость между наличием детей и возвратом кредита в срок?</a></span></li><li><span><a href="#Есть-ли-зависимость-между-семейным-положением-и-возвратом-кредита-в-срок?" data-toc-modified-id="Есть-ли-зависимость-между-семейным-положением-и-возвратом-кредита-в-срок?-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Есть ли зависимость между семейным положением и возвратом кредита в срок?</a></span></li><li><span><a href="#Есть-ли-зависимость-между-уровнем-дохода-и-возвратом-кредита-в-срок?" data-toc-modified-id="Есть-ли-зависимость-между-уровнем-дохода-и-возвратом-кредита-в-срок?-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Есть ли зависимость между уровнем дохода и возвратом кредита в срок?</a></span></li><li><span><a href="#Как-разные-цели-кредита-влияют-на-его-возврат-в-срок?" data-toc-modified-id="Как-разные-цели-кредита-влияют-на-его-возврат-в-срок?-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Как разные цели кредита влияют на его возврат в срок?</a></span></li></ul></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

## Откроем файл с данными и изучим общую информацию

In [2]:
# загружаем датасет
data = pd.read_csv('datasets/data.csv')

# выводим общую информацию
data.info()

# посмотрим на первые 10 строк
display(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
# оценим количество полных дубликатов
print(
    'В датасете {} полных дубликатов'.format(data.duplicated().sum())
)

В датасете 54 полных дубликатов


**Вывод**

- В наборе данных 4 количественных и 8 качественных категорий;
- столбец `days_employed` точно содержит пропуски данных, а также артефакты (отрицательные значения и неестественно большие значения); данные в столбце имеют тип float64, что для нашей задачи избыточно - нам достаточно целых значений общего количества дней трудового стажа;
- стобец `education` содержит неявные дубликаты данных - значения, записанные в разном регистре;
- столбцы `education` и `education_id` а также `family_status` и `family_status_id` дублируют друг друга смыслово
- столбец `total_income` точно содержит пропуски данных; данные в столбце имеют тип float64, что для нашей задачи избыточно - нам достаточно целых значений дохода;
- столбцы `days_employed` и `total_income` содержат одинаковое количество пропусков. Можно сделать предположение, что пропуски данных есть в одних и тех же записях;
- столбец `purpose` содержит множество схожих по смыслу целей кредита, записанных лексически разно. Эти данные предстоит лемматизировать;
- процент повреждённых данных: `(21525 - 19351) / 21525 * 100% = 2174 / 21525 * 100% = 10,1%`. Это довольно значительный объём данных, об этом стоит сообщить коллегам, которые их нам предоставили;
- во всех столбцах, кроме `days_employed` и `total_income` также могут содержаться пропуски, несмотря на то, что там нет значений типа NaN; это нужно будет проверить.

## Предобработка данных

### Обработка пропусков <a id='fill_gaps'></a>

Для начала изучим каждый столбец более тщательно и узнаем о них следующее:

1. Минимум и максимум (для количественных категорий)
2. Список уникальных значений (для качественных категорий)

По каждому типу категорий создадим сводную таблицу для простоты изучения

In [4]:
# отобразим сводную таблицу по количественным категориям
display(
    data[['children', 'days_employed', 'dob_years', 'total_income']].describe().T
)

# отобразим информацию по качественным категориям
for i in ['education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'purpose']:
    print(
        data[i].value_counts()
    )
    print(
        '\n'
    )

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64


женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64


F      14236
M       7288
XNA        1
Name: gender, dtype: int64


сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий    

В основном мы подтвердили первоначальные выводы, но узнали и кое-что новое:

1. Количественные категории 
 - `children`: имеется артефакт в виде минимального значения -1. Посмотрим на такие записи подробнее чуть позже и решим, что с этим делать, так как в контексте возможных значений столбца это можно приравнять к пропуску данных (количество детей не может быть отрицательным)
 - `days_employed`: подтвердили артефакты - отрицательный и неестественно большой стажи
 - `dob_years`: имеет артефакт в виде минимального значения 0. Посмотрим на такие записи подробнее чуть позже и решим, что с этим делать, так как в контексте возможных значений столбца это можно приравнять к пропуску данных (возраст заёмщика не может быть менее 18 лет)
 - `total_income`: минимум и максимум в норме
2. Качественные категории:
 - `education`: много дубликатов. Дубликаты отличаются регистром. При их обработке можем просто перевести все значения в нижний регистр, это решит проблему
 - `education_id`: дубликатов нет
 - `family_status`: дубликатов нет, желательно перевести все в нижний регистр (выбивается заглавная Н в значении "Не женат / не замужем")
 - `family_status_id`: дубликатов нет
 - `gender`: имеется артефакт - значение XNA. Так как усреднить пол или заполнить медианой мы не можем, оставим эти значения без изменения, благо что на поставленные задачи данный столбец не влияет
 - `income_type`: дубликатов нет
 - `debt`: дубликатов нет
 - `purpose`: однозначно лемматизировать

Для начала проверим предположение, которое мы сделали в выводе предыдущего шага. Для этого выберем только те строки, где есть пропуски одновременно и в столбце `days_employed` и в столбце `total_income`. Затем изучим информацию о полученном наборе

In [5]:
data[(data['days_employed'].isnull()) & (data['total_income'].isnull())].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


В полученном датасете содержится 2174 записи, что совпадает с числом записей с пропусками в столбцах `days_employed` и `total_income`. Таким образом, наша изначальная гипотеза была верна, и пропуски данных по столбцам `days_employed` и `total_income` содержатся в одних и тех же записях. Учитывая это, можно сделать предположение, что природа пропусков в этих столбцах - чисто техническая и пропуски имеют в целом случайных характер, что позволяет нам применить метод заполнения медианой для этих столбцов. Сделаем это

#### Обработка пропусков в столбце `days_employed`

Чтобы заполнить пропуски медианой, необходимо избавиться от артефактов.
1. Исходя из предположения, что отрицательные значения это чисто техническая опечатка, сначала обработаем столбец функцией `abs()`, чтобы убрать отрицательные значения.
2. Так как мы точно не знаем природу появления записей с аномально большим стажем, мы можем лишь сделать предположение о том, почему они имеют такие большие значения. Предположим, что наши аномалии - это ошибка конвертации при обработке анкет и данные в этих записях указаные не в днях, а часах. Для обработки этих значений напишем функцию, которая будет проверять переданное ей значение, и, в случае, если оно занимает имеет шесть или более разрядов, будем делить его на 24. Затем применим эту функцию к столбцу

In [6]:
# убираем отрицательные значения
data['days_employed'] = data['days_employed'].apply(abs)

# пишем функцию
def trim_days_employed(row):
    if row['days_employed'] / 100000 >= 1:
        return round(row['days_employed'] / 24)
    
# применим фукнцию к столбцу
data['days_employed'] = data.apply(trim_days_employed, axis = 1)

# проверим результат работы функции - изучим полученные значения столбца 'days_employed'
display(data.days_employed.describe())

# рассчитаем медианный стаж
med_days_employed = data['days_employed'].median()

# посмотрим на него, чтобы не было неожиданностей
print(f'Медианное значение стажа: {med_days_employed}', '\n')

# заполняем пропуски медианой
data = data.fillna(value = {'days_employed' : med_days_employed})

# проверяем общую информацию о наборе данных
data.info()

count     3445.000000
mean     15208.510885
std        878.132912
min      13697.000000
25%      14443.000000
50%      15217.000000
75%      15969.000000
max      16740.000000
Name: days_employed, dtype: float64

Медианное значение стажа: 15217.0 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пропуски данных в столбце `days_employed` успешно заполнены. Теперь избавимся от артефактов (расцениваемых как пропуски) в столбце `dob_years`.

#### Обработка пропусков в столбце `dob_years`

Узнаем, в каких категориях дохода содержатся такие записи и сохраним их в отдельный список.

In [7]:
# сохраняем список категорий
zero_years = list(data[data['dob_years'] == 0]['income_type'].unique())

# посмотрим на них более детально
display(zero_years)

['пенсионер', 'сотрудник', 'компаньон', 'госслужащий']

Логично по каждой категории заполнить нули медианным значением для данной категории. Перебором по списку `zero_years` в цикле заполним нулевой возраст в каждой категории медианным значением возраста в данной категории. При расчёте медианы будем учитывать только ненулевые значения

In [8]:
# заполняем нули медианой по всем категориям из списка с помощью цикла
for i in zero_years:
    data.loc[(data['dob_years'] == 0) & (data['income_type'] == i), 'dob_years'] = int(data[(data['income_type'] == i) & (data['dob_years'] > 0)]['dob_years'].median())
    print(int(data[(data['income_type'] == i) & (data['dob_years'] > 0)]['dob_years'].median()))

# проверим, как изменились значения    
display(data.groupby('income_type').agg({'dob_years' : ['min', 'max']}))

60
39
39
40


dob_years    
                      min max
income_type                  
безработный            31  45
в декрете              39  39
госслужащий            19  75
компаньон              19  74
пенсионер              22  74
предприниматель        27  58
сотрудник              19  74
студент                22  22

Отлично, заёмщиков с нулевым возрастом больше нет. Теперь разберёмся со столбцом `children`.

#### Обработка пропусков в столбце `children`

Так как мы предполагаем, что значение -1 в этом поле - это техническая ошибка, можно просто убрать знак `-` с помощью функции `abs` 

In [9]:
data['children'] = data['children'].apply(abs)

Отлично, теперь отрицательное количество детей не будет мешать нам. Пора заняться столбцом `total_income`.

#### Обработка пропусков в столбце `total_income`

К нему применим те же действия, что и к столбцу `dob_years` - сначала получим список категорий, для которых есть пропуски в значениях дохода, а потом уже заполним эти пропуски медианными значениями для каждой из категорий.

In [10]:
# сохраняем список категорий
missed_income = list(data[data['total_income'].isnull()]['income_type'].unique())

# посмотрим на них
print(missed_income)

['пенсионер', 'госслужащий', 'компаньон', 'сотрудник', 'предприниматель']


Перебором по списку `missed_income` в цикле заполним нулевой возраст в каждой категории медианным значением возраста в данной категории. Значения типа `NaN` при расчёте медианы не учитываются, поэтому можно не писать дополнительных условий

In [11]:
# сначала проверим, корректно ли рассчитываются медианы по всем категориям
for i in missed_income:
    print('Медиана в категории {}: {}'.format(i, data[data['income_type'] == i]['total_income'].median()))

Медиана в категории пенсионер: 118514.48641164352
Медиана в категории госслужащий: 150447.9352830068
Медиана в категории компаньон: 172357.95096577113
Медиана в категории сотрудник: 142594.39684740017
Медиана в категории предприниматель: 499163.1449470857


In [12]:
# заполняем нули медианой по всем категориям из списка с помощью цикла
for i in missed_income:
    data.loc[(data['total_income'].isnull()) & (data['income_type'] == i), 'total_income'] = data[data['income_type'] == i]['total_income'].median()

# проверим, как изменились значения    
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Ура! Мы избавились от пропусков и существенных артефактов! Настало время привести значения столбцов `days_employed` и `total_income` к целочисленному типу.

P.S. Осталось ещё значение `Не женат / не замужем`, которое раздражает своей заглавной буквой. Исправим эту досадную мелочь:

In [13]:
data['family_status'] = data['family_status'].str.lower()

#### Вывод

При изучении данных мы обнаружили как явные пропуски данных (значения типа `NaN` в столбцах `days_employed` и `total_income`), так и неявные - нехарактерные значения в остальных столбцах, которые в контексте этих данных можно расценивать как пропуски - нулевой возраст, отрицательное количество детей, пол типа `XNA`. Мы сделали предположение о причине появления этих пропусков - техническая ошибка при подготовке данных. Для обработки пропусков мы выбрали метод заполнения медианой, т.к. наши пропуски являются абсолютно случайными.

### Замена типа данных

Сделаем вид, что мы не знаем исходного типа данных в этих столбцах. Если предположить, что столбцы могут содержать как числовые данные, так и строковые их представления, то применив к ним функцию `to_numeric()` мы получим данные типа `int64` тогда и только тогда, когда все изначальные данные были целыми числами. Если хотя бы одно значение будем иметь мантиссу, мы получим столбец вещественных значений, что нам категорически не подходит. Поэтому верным варинтом будет применить функцию `astype()` с параметром `int`. С учётом предположений о строковых представлениях числовых данных в исходном столбце будет правильно нашу замену поместить внутрь конструкции `try - except`, чтобы в случае некорректных входных данных мы получили предупреждение о необходимости проверки входных данных, а не упавший код. Сделаем это:

In [14]:
# попробуем изменить тип данных в столбце days_employed
try:
    data['days_employed'] = data['days_employed'].astype(int)
except:
    print('Проверь входные данные в столбце days_employed!')
    
# попробуем изменить тип данных в столбце total_income
try:
    data['total_income'] = data['total_income'].astype(int)
except:
    print('Проверь входные данные в столбце total_income!')

# проверим, как прошла конверсия    
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int32
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int32
purpose             21525 non-null object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB
None


Прекрасно! Больше данных вещественного типа у нас нет!

**Вывод**

Мы изменили тип данных в столбцах `days_employed` и `total_income` с вещественного на целочисленный. Для этого мы сначала обосновали выбор метода изменения данных, а потом произвели само изменение данных, применив конструкцию `try - except` для обработки возможных ошибок ввода, если входные данные вдруг поменяются.

### Обработка дубликатов

Для начала посмотрим, сколько у нас на данный момент дубликатов:

In [15]:
print(f'Количество дубликатов в таблице: {data.duplicated().sum()}')

Количество дубликатов в таблице: 54


На этом можно было бы остановиться, но мы ранее выяснили, что в столбце `education` у нас имеются дубликаты категорий. Из-за этого дубликаты могут быть найдены не полностью - при совпадении всех остальных столбцов дубликатом не будут считаться записи, у которых, к примеру в столбце `education` записано `Высшее` и `высшее`, соответственно.

Так как мы выяснили, что дублирующиеся категории отличаются только регистром, логичным кажется применить к этому столбцу функцию `str.lower()`. Но сначала надо убедиться, что все категории соответствуют своим числовым дублёрам из столбца `education_id`. Давайте посмотрим:

In [16]:
display(data.groupby('education_id').agg({'education' : 'value_counts'}))

education
education_id education                     
0            высшее                    4718
             ВЫСШЕЕ                     274
             Высшее                     268
1            среднее                  13750
             СРЕДНЕЕ                    772
             Среднее                    711
2            неоконченное высшее        668
             Неоконченное высшее         47
             НЕОКОНЧЕННОЕ ВЫСШЕЕ         29
3            начальное                  250
             НАЧАЛЬНОЕ                   17
             Начальное                   15
4            ученая степень               4
             УЧЕНАЯ СТЕПЕНЬ               1
             Ученая степень               1

Как видно из таблицы, каждому `education_id` соответствует только один тип образования, но в разных написаниях. Поэтому можем смело перевести значения столбца `education` в нижний регистр:

In [17]:
# переводим значения в нижний регистр
data['education'] = data['education'].str.lower()

# ещё раз выведем сводную таблицу и посмотрим на изменения
display(data.groupby('education_id').agg({'education' : 'value_counts'}))

,,education
education_id,education,
0,высшее,5260
1,среднее,15233
2,неоконченное высшее,744
3,начальное,282
4,ученая степень,6


Отлично. Теперь каждому `education_id` соответствует свой тип образования ровно в одном написании. Что-то подсказывает, что количество дубликатов теперь увеличится. Проверим:

In [18]:
print(f'Количество дубликатов в таблице: {data.duplicated().sum()}')

Количество дубликатов в таблице: 71


Из-за дублирования написания категорий мы могли не обработать целых 17 значений, а это почти 24% от общего количества дубликатов. Пора распрощаться с дубликатами:

In [19]:
data = data.drop_duplicates().reset_index(drop = True)

**Вывод**

Мы выяснили, сколько всего дубликатов в данных. С учётом сведений, полученных ранее, мы предположили, что список дубликатов может быть неполным. После устранения дублирующих друг друга значений в столбце `education` и повторного запроса количества дубликатов, мы выяснили, что количество дубликатов на самом деле больше, чем было получено изначально. После этого мы удалили все дубликаты из набора данных. Говоря о причинах, можно предположить, что полные дубликаты - это результат технического сбоя, а дубликаты наименований категорий образования - это следствие человеческого фактора - при заполнении анкеты каждый заполнял её, как считал удобным.

### Лемматизация

Сначала нам нужно определить, сколько всего у нас будет категорий займа. Для этого возьмём все уникальные значения столбца `purpose`, объединим в одну строку, лемматизируем и посчитаем количество каждой из лемм:

In [20]:
m = Mystem()

# записываем в одну строчку все уникальные значения целей кредита из основного набора данных
purpose_unique = ' '.join(list(data['purpose'].unique()))

# заодно посмотрим, сколько у нас всего уникальных значений
print('Всего уникальных значений в столбце purpose:', len(list(data['purpose'].unique())), '\n')

# лемматизируем
purpose_lemmas = m.lemmatize(purpose_unique)

# изучим полученные леммы и их частоту появления
print(Counter(purpose_lemmas))

Всего уникальных значений в столбце purpose: 38 

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


Изучив полученные данные, можно выделить следующие основные цели займа:

- Покупка недвижимости (леммы "жильё", "недвижимость")
- Покупка автомобиля (лемма "автомобиль")
- Получение образования (лемма "образование")
- Проведение свадьбы (лемма "свадьба")
- Стройтельство недвижимости (лемма "строительство")
- Ремонт недвижимости (лемма "ремонт")

Для того, чтобы привести все имеющиеся значения столбца `purpose` к этому списку необходимо сделать следующее:

- Создать новый датафрейм, в который поместить уникальные значения столбца `purpose`
- Написать функцию, которая принимает на вход строку, лемматизирует её и проверяет вхождение определённых для каждой категории лемм в полученный список, после чего возвращает одну из категорий
- Создать в новом датафрейме новый столбец, применив данную функцию к существующему столбцу. Таким образом каждому из большого количество уникальных значений столбца `purpose` будет сопоставлена одна из 6 определённых нами целей займа
- С помощью функции `merge` объединить основной датафрейм и созданный нами побочный

Применять нашу функцию только к уникальным значениям необходимо в связи с тем, что лемматизация это очень долгий и ресурсозатратный процесс.

После этого также с помощью просто функции добавим к датафрейму столбец с цифровыми индексами для каждого типа займа. Для этого сначала составим словарь вида `{'цель_займа' : 'индекс'}`, а после в теле функции будем возвращать значения нового столбца по ключу.

In [21]:
# создаём новый датафрейм

p_l = pd.DataFrame({'purpose' : data['purpose'].unique()})

# напишем нашу функцию
def lemm_cat(row):
    s = m.lemmatize(row['purpose'])
    if ('жилье' in s) or ('недвижимость' in s):
        if 'ремонт' in s:
            return 'Займ на ремонт недвижимости'
        elif 'строительство' in s:
            return 'Займ на строительство недвижимости'
        else:
            return 'Займ на покупку недвижимости'
    if 'автомобиль' in s:
        return 'Займ на покупку автомобиля'
    if 'образование' in s:
        return 'Займ на получение образования'
    if 'свадьба' in s:
        return 'Займ на проведение свадьбы'

# применяем функцию    
p_l['purpose_cat'] = p_l.apply(lemm_cat, axis=1)

# создадим словарь

purpose_cat_ids = {'Займ на покупку недвижимости' : 0,
                  'Займ на покупку автомобиля' : 1,
                  'Займ на получение образования' : 2,
                  'Займ на проведение свадьбы' : 3,
                  'Займ на строительство недвижимости' : 4,
                  'Займ на ремонт недвижимости' : 5}

# напишем функцию для присвоения цифрового индекса
def purpose_id(row):
    return purpose_cat_ids[row['purpose_cat']]

# добавим цифровой индекс для каждой цели займа
p_l['purpose_cat_id'] = p_l.apply(purpose_id, axis=1)

# объединяем датафреймы
data = data.merge(p_l, on = 'purpose')

# посмотрим на объединённые данные
display(data.sort_values(by = 'total_income', ascending = False).head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_cat,purpose_cat_id
20704,0,15217,44,высшее,0,женат / замужем,0,M,компаньон,0,2265604,ремонт жилью,Займ на ремонт недвижимости,5
8549,1,15217,39,высшее,0,женат / замужем,0,M,компаньон,1,2200852,строительство недвижимости,Займ на строительство недвижимости,4
1295,1,15217,35,среднее,1,гражданский брак,1,M,сотрудник,0,1726276,дополнительное образование,Займ на получение образования,2
4812,0,15217,61,среднее,1,не женат / не замужем,4,F,сотрудник,0,1715018,покупка жилья для семьи,Займ на покупку недвижимости,0
2183,0,15217,42,высшее,0,гражданский брак,1,M,компаньон,0,1711309,сыграть свадьбу,Займ на проведение свадьбы,3


Прекрасно! Вместо 38 изначальных целей займа, дублировавших друг друга по смыслу, у нас теперь 6 понятных категорий, охватывающих все изначальные цели.

**Вывод**

Ранее мы заметили, что в столбце, содержащем цели займа у нас содержиться множество схожих смыслово, но разных по написанию целей. Это помешало бы нам решить одну из задач исследования, поэтому мы должны были выделить основные цели займа и добавить их к набору данных, чтобы в дальнейшем нам было удобнее группировать данные и отслеживать зависимости. Для этого мы:
- Лемматизировали строку, состоящую из всех уникальных значений столбца `purpose`
- Посчитали количество каждой из полученных лемм
- Выделили на основании полученных данных 6 основных категорий займа
- Присвоили этим категориям цифровые индексы
- С помощью функции сопоставили каждой из оригинальных целей одну из шести полученных категорий и соответствующий цифровой индекс
- Добавили полученные значения в основные данные

### Категоризация данных

Для того, чтобы ответить на один из вопросов, поставленных перед нами в проекте, а именно "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?" нам необходимо эти самые уровни дохода определить, то есть категоризировать данные по столбцу `total_income`. Для этого сначала обозначим предполагаемые категории дохода и их цифровые индексы, а также границы каждой из категорий. Границы будем определять в процентном соотношении от размаха данных по столбцу `total_income`. Чтобы получить равнорелевантные категории, определим их границы через процентили. Предполагаемых категорий будет 4:

0 - Невысокий доход - [min, 25-й процентиль]<br>
1 - Средний доход - [25-й процентиль, 50-й процентиль]<br>
2 - Высокий доход - [50-й процентиль, 75-й процентиль]<br>
3 - Сверхвысокий доход - [75-й процентиль, max]

Для категоризации необходимо будет:

- Посчитать границы категорий
- Составить словарь с определениями наших категорий (индекс, название)
- Написать функцию, которая будет проверять вхождение аргумента в одну из категорий и возвращать её наименование
- Применить эту функцию к столбцу `total_income` и получить новый столбец `total_income_cat`
- Написать функцию, которая будет возвращать индекс категории по её имени из словаря
- Применить эту функцию к столбцу `total_income_cat` и получить новый столбец `total_income_cat_id`

In [22]:
# смотрим на данные и записываем результат работы describe() в отдельный список.
#  Он нам ещё пригодится, чтобы не пересчитывать значения каждый раз
total_income_decription = list(data.total_income.describe())
display(data.total_income.describe())

# минимум, максимум
min = total_income_decription[3]
max = total_income_decription[7]

# правая граница первой категории
b_25 = total_income_decription[4]

# правая граница второй категории
b_50 = total_income_decription[5]

# правая граница третьей категории
b_75 = total_income_decription[6]

# создаём словарь

total_income_cat_ids = {'Невысокий доход' : 0,
                       'Средний доход' : 1,
                       'Высокий доход' : 2,
                       'Сверхвысокий доход' : 3}

# пишем функцию для присвоения категории
def income_cat(row):
    s = row['total_income']
    if s <= b_25:
        return 'Невысокий доход'
    if s <= b_50:
        return 'Средний доход'
    if s <= b_75:
        return 'Высокий доход'
    if s <= max:
        return 'Сверхвысокий доход'

# получаем столбец с категориями дохода
data['total_income_cat'] = data.apply(income_cat, axis=1)

# пишем функцию для получения цифрового индекса
def income_cat_id(row):
    return total_income_cat_ids[row['total_income_cat']]

# смотрим на то, что получилось

display(data.head())    

count    2.145400e+04
mean     1.653196e+05
std      9.818730e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.425940e+05
75%      1.958202e+05
max      2.265604e+06
Name: total_income, dtype: float64

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_cat,purpose_cat_id,total_income_cat
0,1,15217,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,Займ на покупку недвижимости,0,Сверхвысокий доход
1,0,15217,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,Займ на покупку недвижимости,0,Высокий доход
2,0,15217,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,Займ на покупку недвижимости,0,Сверхвысокий доход
3,0,15217,48,среднее,1,женат / замужем,0,F,компаньон,0,157245,покупка жилья,Займ на покупку недвижимости,0,Высокий доход
4,0,15217,41,среднее,1,женат / замужем,0,M,госслужащий,0,118551,покупка жилья,Займ на покупку недвижимости,0,Средний доход


**Вывод**

Чтобы ответить на один из вопросов, поставленных нам в задаче, мы категоризировали данные по уровню дохода. Для этого мы:

- Определили количество категорий, их границы, сопоставили им цифровые индексы
- Написали функцию, которые присваивают категорию дохода в зависимости от его значения
- Составили словарь с названиями категорий и их цифровыми индексами
- Написали функцию, возвращающую цифровой индекс категории по её названию
- Применили эти функции последовательно и получили в данных новые столбцы с категорией дохода и цифровым индексом данной категории

## Ответим на интересующие банк вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на этот вопрос:

- Сгруппируем данные по столбцу `children` и применим к столбцу `debt` функции `sum` и `count` для получения числа должников и общего числа записей в группе соответственно. Эти данные запишем в новый датафрейм
- Добавим в полученный датафрейм ещё один столбец, в который запишем отношение числа должников к общему числу в группе и умножим на 100. Результат округлим до 2 чисел после запятой. Таким образом получим процент должников в каждой группе

In [23]:
# создаём новый датафрейм с группировкой
child_data = data.groupby('children').agg({'debt' : ['sum', 'count']})

# добавляем столбец с процентом должников
child_data['debt_ratio'] = round(child_data['debt']['sum'] / child_data['debt']['count'] * 100, 2)

# изучаем
display(child_data.sort_values(by = 'debt_ratio', ascending = False))

debt        debt_ratio
           sum  count           
children                        
20           8     76      10.53
4            4     41       9.76
2          194   2052       9.45
1          445   4855       9.17
3           27    330       8.18
0         1063  14091       7.54
5            0      9       0.00

**Вывод**

Исходя из полученных данных можно сделать вывод, что зависимость между наличием детей и возвратом кредита в срок существует. Заёмщики без детей имеют самый низкий процент должников (не считая аномальной группы заёмщиков с 5-ю детьми, где должников нет вовсе). С увеличением числа детей процент должников в группе начинает возрастать (опять же за исключением групп с 3-мя и 5-ю детьми). В целом это логично - чем больше детей, тем выше финансовая нагрузка и выше риск невозврата займа в срок. 

Процент по категориям, в которых трое и более детей, может быть недостоверным, так как выборки довольно малы и потому нерепрезентативны</div>

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на этот вопрос:

- Сгруппируем данные по столбцу `family_status` и применим к столбцу `debt` функции `sum` и `count` для получения числа должников и общего числа записей в группе соответственно. Эти данные запишем в новый датафрейм
- Добавим в полученный датафрейм ещё один столбец, в который запишем отношение числа должников к общему числу в группе и умножим на 100. Результат округлим до 2 чисел после запятой. Таким образом получим процент должников в каждой группе

In [24]:
# создаём новый датафрейм с группировкой
family_data = data.groupby('family_status').agg({'debt' : ['sum', 'count']})

# добавляем столбец с процентом должников
family_data['debt_ratio'] = round(family_data['debt']['sum'] / family_data['debt']['count'] * 100, 2)

# изучаем
display(family_data.sort_values(by = 'debt_ratio', ascending = False))

debt        debt_ratio
                       sum  count           
family_status                               
не женат / не замужем  274   2810       9.75
гражданский брак       388   4151       9.35
женат / замужем        931  12339       7.55
в разводе               85   1195       7.11
вдовец / вдова          63    959       6.57

**Вывод**

На основе полученных данных можно сделать вывод о том, что зависимость между семейным положением и возвратом кредита в срок существует. Описать её можно следующим образом - при каждом переходе на следующую стадию семейного положения процент должников снижается.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для ответа на этот вопрос:

- Сгруппируем данные по столбцу `total_income_cat` и применим к столбцу `debt` функции `sum` и `count` для получения числа должников и общего числа записей в группе соответственно. Эти данные запишем в новый датафрейм
- Добавим в полученный датафрейм ещё один столбец, в который запишем отношение числа должников к общему числу в группе и умножим на 100. Результат округлим до 2 чисел после запятой. Таким образом получим процент должников в каждой группе

In [25]:
# создаём новый датафрейм с группировкой
income_data = data.groupby('total_income_cat').agg({'debt' : ['sum', 'count']})

# добавляем столбец с процентом должников
income_data['debt_ratio'] = round(income_data['debt']['sum'] / income_data['debt']['count'] * 100, 2)

# изучаем
display(income_data.sort_values(by = 'debt_ratio', ascending = False))

# data

debt       debt_ratio
                    sum count           
total_income_cat                        
Средний доход       483  5479       8.82
Высокий доход       448  5247       8.54
Невысокий доход     427  5364       7.96
Сверхвысокий доход  383  5364       7.14

**Вывод**

Меньше всего должников в категории со сверхвысоким доходом (7,14 %). Это вполне логично - наличие сверхвысокого дохода позволяет поддерживать финансовую дисциплину.
    
Следующая по возрастанию категория должников (7,96 %) - обладатели невысокого дохода. Тут в целом также есть логика. Люди с таким доходам стараются выплачивать кредиты вовремя, чтобы избежать начисления штрафов и пеней, а также сохранить (или создать) себе хорошую кредитную историю
    
Остальные категории дохода (средний и высокий) имеют примерно равный процент должников (8,82 % и 8,54 % соответственно). Предположим, что это можно объяснить либо тем, что данным категориям уже не так страшны штрафы и пени в силу более высокого дохода.
    
Резюмируя, можно сказать, что зависимость между уровнем дохода и возвратом кредита в срок существует

### Как разные цели кредита влияют на его возврат в срок?

Для ответа на этот вопрос:

- Создадим сводную таблицу, сгруппированную по целям кредита. В качестве значений, по которым мы хотим увидеть сводку, возмьём значения столбца `debt`. В качестве столбца, по которому будет проводится горизонтальная группировка, выберем категорию дохода `total_income_cat`. К значениям применим функции `sum` и `count`. Таким образом мы получим сводную таблицу с количеством должников, сгруппированных по целям займа и категориям дохода, а также с общим количеством заёмщиков сгруппированным таким же образом. Эти данные запишем в новый датафрейм
- Добавим в полученный датафрейм ещё один столбец, в который запишем отношение числа должников к общему числу в группе и умножим на 100. Результат округлим до 2 чисел после запятой. Таким образом получим процент должников в каждой группе

In [26]:
# создадим новый датафрейм, куда сохраним сводную таблицу по должникам
purpose_data = data.pivot_table(index = 'purpose_cat', values = 'debt', aggfunc = ['sum', 'count', 'mean'])
purpose_data['mean'] = round(purpose_data['mean'] * 100, 2)

# изучаем
display(
    purpose_data
)

,sum,count,mean
,debt,debt,debt
purpose_cat,,,
Займ на покупку автомобиля,403,4306,9.36
Займ на покупку недвижимости,603,8326,7.24
Займ на получение образования,370,4013,9.22
Займ на проведение свадьбы,186,2324,8.00
Займ на ремонт недвижимости,35,607,5.77
Займ на строительство недвижимости,144,1878,7.67


**Вывод**

В общем и целом также можно проследить зависимость между целью кредита и процентом невозврата в срок:
- Самый низкий процент просрочек (5,77 %) - среди займов на ремонт. Как правило ремонт предполагает наличие чёткой сметы и сравнительно небольшой суммы, поэтому и так мало невозвратов в срок
- Покупка и строительство недвижимости (7,24 % и 7,67 %) - серьёзные шаги, которые также предполагают серьёзный предварительный расчёт, но и намного большие суммы по сравнению с ремонтом, следовательно и намного большие ежемесячные платежи, и более высокий процент просрочек
- Свадьба (8 %) - в России кредит на свадьбу как правило берётся с расчётом окупить его подарками от гостей и родственников, но так как это величина практически случайная, зачастую расчёт молодожёнов не оправдывается. Отсюда и высокий процент просрочек, несмотря на вроде бы более скромную цель, чем строительство или покупка недвижимости.
- Займ на получение образования (9,22 %) - повышенный процент просрочек предположительно связан с тем, что займ берут в надежде получить новое образование и новый скачок в карьере, но выполнить скачок получается сразу не у всех, откуда и просрочки платежей
- Автомобиль (9,36 %) - самый логичный лидер по числу просрочек с учётом специфики нашего общества. Многие берут автомобили в кредит абсолютно не по средствам, откуда и максимальный процент просрочек.

## Общий вывод

***Есть ли зависимость между наличием детей и возвратом кредита в срок?***

**Да, зависимость существует.**
Исходя из полученных данных можно сделать вывод, что зависимость между наличием детей и возвратом кредита в срок существует. Заёмщики без детей имеют самый низкий процент должников (не считая аномальной группы заёмщиков с 5-ю детьми, где должников нет вовсе) - 7,54 %. С увеличением числа детей процент должников в группе начинает возрастать (опять же за исключением групп с 3-мя и 5-ю детьми) - 9,17 % для заёмщиков с 1 ребёнком, 9,45 % для заёмщиков с 2-мя детьми. В целом это логично - чем больше детей, тем выше финансовая нагрузка и выше риск невозврата займа в срок. Процент по категориям, в которых трое и более детей, может быть недостоверным, так как выборки довольно малы и потому нерепрезентативны (3 - 8,18 %, 4 - 9,76 %, 5 - 0 %, 20 - 10,53 %).
<br><br><br><br>
***Есть ли зависимость между семейным положением и возвратом кредита в срок?***

**Да, зависимость существует.** Наибольший процент просрочек у холостых людей - 9,75 %. С каждым переходом в новую категорию семейного положения процент снижается. У состоящих в гражданском браке уже 9,35 %, у людей в официальном браке процент просрочек существенно снижается - до 7,55 %, среди окончивших свой брак разводом просрочек всего 7,11 %, и, наконец, минимальное количество просрочек допускают овдовевшие заёмщики - 6,57  %.
<br><br><br><br>
***Есть ли зависимость между уровнем дохода и возвратом кредита в срок?***

**Да, зависимость существует.** Меньше всего должников в категории со сверхвысоким доходом (7,14 %). Это вполне логично - наличие сверхвысокого дохода позволяет поддерживать финансовую дисциплину. Следующая по возрастанию категория должников (7,96 %) - обладатели невысокого дохода. Тут в целом также есть логика. Люди с таким доходам стараются выплачивать кредиты вовремя, чтобы избежать начисления штрафов и пеней, а также сохранить (или создать) себе хорошую кредитную историю. Остальные категории дохода (средний и высокий) имеют примерно равный процент должников (8,82 % и 8,54 % соответственно). Предположим, что это можно объяснить либо тем, что данным категориям уже не так страшны штрафы и пени в силу более высокого дохода.
<br><br><br><br>
***Как разные цели кредита влияют на его возврат в срок?***

Самый низкий процент просрочек (5,77 %) - среди займов на ремонт. Как правило ремонт предполагает наличие чёткой сметы и сравнительно небольшой суммы, поэтому и так мало невозвратов в срок. Покупка и строительство недвижимости (7,24 % и 7,67 %) - серьёзные шаги, которые также предполагают серьёзный предварительный расчёт, но и намного большие суммы по сравнению с ремонтом, следовательно и намного большие ежемесячные платежи, и более высокий процент просрочек. Свадьба (8 %) - в России кредит на свадьбу как правило берётся с расчётом окупить его подарками от гостей и родственников, но так как это величина практически случайная, зачастую расчёт молодожёнов не оправдывается. Отсюда и высокий процент просрочек, несмотря на вроде бы более скромную цель, чем строительство или покупка недвижимости.Займ на получение образования (9,22 %) - повышенный процент просрочек предположительно связан с тем, что займ берут в надежде получить новое образование и новый скачок в карьере, но выполнить скачок получается сразу не у всех, откуда и просрочки платежей. Автомобиль (9,36 %) - самый логичный лидер по числу просрочек с учётом специфики нашего общества. Многие берут автомобили в кредит абсолютно не по средствам, откуда и максимальный процент просрочек. 